Create connection and parse first message

In [61]:
import socket
IP = '127.0.0.1'
PORT = 4242
HANDSHAKE = 'READY'

# ? Create socket
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
sock.sendto(HANDSHAKE.encode('UTF-8'), (IP, PORT))

# TODO Loop other messages

# ? Skip garbage
while (data != 'MSG_START'):
	data, addr = sock.recvfrom(1024)
	data = data.decode('UTF-8')

# ? Parse position
data, addr = sock.recvfrom(1024)
data = data.decode('UTF-8')
dt = data[data.find('\n')+1:]
posX = dt[:dt.find('\n')]
dt = dt[dt.find('\n')+1:]
posY = dt[:dt.find('\n')]
dt = dt[dt.find('\n')+1:]
posZ = dt[:dt.find('\n')]
# print('posX: ', posX)
# print('posY: ', posY)
# print('posZ: ', posZ)

# ? Parse Speed
data, addr = sock.recvfrom(1024)
data = data.decode('UTF-8')
dt = data[data.find('\n')+1:]
s = dt[:dt.find('\n')]
# print('Speed: ', s)

# ? Parse Acceleration
data, addr = sock.recvfrom(1024)
data = data.decode('UTF-8')
dt = data[data.find('\n')+1:]
aX = dt[:dt.find('\n')]
dt = dt[dt.find('\n')+1:]
aY = dt[:dt.find('\n')]
dt = dt[dt.find('\n')+1:]
aZ = dt[:dt.find('\n')]
# print('aX: ', aX)
# print('aY: ', aY)
# print('aZ: ', aZ)

# ? Parse Direction
data, addr = sock.recvfrom(1024)
data = data.decode('UTF-8')
dt = data[data.find('\n')+1:]
dX = dt[:dt.find('\n')]
dt = dt[dt.find('\n')+1:]
dY = dt[:dt.find('\n')]
dt = dt[dt.find('\n')+1:]
dZ = dt[:dt.find('\n')]
# print('dX: ', dX)
# print('dY: ', dY)
# print('dZ: ', dZ)

Predict future position from parsed data

In [ ]:

# TODO Convert units to match each other

# ? Position
P = [[posX],
     [posY],
     [posZ]]

# ? Acceleration
A = [[aX],
     [aY],
     [aZ]]

# ? Direction
D = [[dX],
     [dY],
     [dZ]]

# ? time delta
# TODO t value TBD
t = 3 # ? in seconds

# ? let be F be the estimated final position matrix
# ? let noise_corr be the correction of the additive noise caused by sensors incertainity
# ? formula should be
# ? F = P + s * D * t + 0.5 * A * t ** 2 + noise_corr(P, s, D, A)